In [1]:
from exp.utils import *
from exp.models import *
from exp.losses import *
from tqdm.notebook import tqdm
from multiprocessing import Pool

import torch
import torch.nn as NN
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [2]:
picked_labels = ["Atelectasis"]#["Atelectasis", "Cardiomegaly", "Pneumonia"]
already_pretrained_labels = ["Atelectasis"]#["Atelectasis"]#["Cardiomegaly", "Pneumonia"]
already_pretrained_lrs = [1e-3]#[1e-8]#[1e-10, 1e-10]
already_pretrained = {k:v for k,v in zip(already_pretrained_labels, already_pretrained_lrs)}
already_trained = []#["Atelectasis"]
train_labels = [l for l in picked_labels if l not in already_trained]

In [3]:
for label in sorted(train_labels):
    print(f"Training model to classify '{label}'")
    
    # Seed
    seed = 92
    seed_everything(seed)
    
    # Inital setup
    model_name = f"DenseNet121_v4_{label}"#f"sam_densenet_v1_{label}"
    model_type = "densenet"
    bs = 16
    lr = 1e-3
    epochs = 50
    patience = 1
    rho = 0.2
    with_reset = False
    image_size = (224, 224)
    device = get_device()
    labels = get_labels()
    
    # Load data
    train_df, valid_df, test_df = get_dataframes(include_labels=labels, 
                                                 small=False)
    print(train_df.shape, valid_df.shape, test_df.shape)
    train_df = get_binary_df(label, train_df)
    valid_df = get_binary_df(label, valid_df)
    test_df = get_binary_df(label, test_df)
    
    # Compute label weights
    train_label = train_df[[label]].values
    neg_weights, pos_weights = compute_class_freqs(train_label)
    neg_weights, pos_weights = torch.Tensor(neg_weights), torch.Tensor(pos_weights)
    print(neg_weights, pos_weights)
    
    # Get transforms
    train_tfs, test_tfs = get_transforms(image_size=image_size)
    
    # Create datasets
    train_ds = CRX8_Data(train_df, get_image_path(), label, image_size=image_size, transforms=train_tfs)
    valid_ds = CRX8_Data(valid_df, get_image_path(), label, image_size=image_size, transforms=test_tfs)
    test_ds  = CRX8_Data(test_df , get_image_path(), label, image_size=image_size, transforms=test_tfs)
    
    # Create dataloaders
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
    valid_dl = DataLoader(valid_ds, batch_size=bs, shuffle=False)
    test_dl  = DataLoader(test_ds,  batch_size=bs, shuffle=False)
    dataloaders = {
        "train": train_dl,
        "val": valid_dl,
        "test": test_dl
    }
    
    # Load imagenet-pretrained model
    if label in list(already_pretrained.keys()):
        model = load_model(model_name)
        lr = already_pretrained[label]
        print("Loaded pretrained model!")
    else:
        model = pretrained_densenet121()
    model = model.to(device)
    
    
    
    # Get criterion and optimizer
    criterion = get_weighted_loss_with_logits(pos_weights.to(device), 
                                              neg_weights.to(device))
    sam_optimizer = SAM(model.parameters(), torch.optim.Adam, lr=lr, rho=rho)
    
    # Train model
    model, history = fit(model, criterion, sam_optimizer, 
                         dataloaders, model_name, epochs, 
                         lr, sam=True, with_reset=with_reset, metric="loss", patience=patience)

FERTIG()

Training model to classify 'Atelectasis'
Using the GPU!
(69219, 25) (17305, 25) (25596, 24)
tensor([0.0961]) tensor([0.9039])
Loaded pretrained model!
Epoch 1:



Train: Loss: 0.107, Acc: 0.619, AUROC: 0.721



Val: Loss: 0.104, Acc: 0.649, AUROC: 0.737
Saved model with loss 0.1038
Epoch 2:



Train: Loss: 0.103, Acc: 0.631, AUROC: 0.749



Val: Loss: 0.102, Acc: 0.669, AUROC: 0.752
Saved model with loss 0.1016
Epoch 3:


...


Val: Loss: 0.098, Acc: 0.665, AUROC: 0.776
Lowered lr to 1.0000000000000002e-12
Epoch 20:



Train: Loss: 0.093, Acc: 0.666, AUROC: 0.806



Val: Loss: 0.098, Acc: 0.691, AUROC: 0.775
Lowered lr to 1.0000000000000002e-13
Epoch 21:



Train: Loss: 0.092, Acc: 0.668, AUROC: 0.808



Val: Loss: 0.098, Acc: 0.657, AUROC: 0.778
Lowered lr to 1.0000000000000002e-14
Learning rate is basically zero. Stopping training.
FERTIG! :D
